In [1]:
!pip3 install spotipy
!pip3 install psycopg2
from sklearn.preprocessing import StandardScaler

import numpy as np
import requests
import joblib
import json
import psycopg2
import pandas as pd
import copy
import spotipy
from tabulate import tabulate


from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from operator import itemgetter
from spotipy.oauth2 import SpotifyClientCredentials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
KEYS = ["danceability",
        "energy",
        "loudness",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence"
        ]

MOOD = {
    0: "Unsure",
    1: "Happy",
    2: "Romantic",
    3: "Sad",
    4: "Focused",
    5: "Energetic"
}

# ["Weather", "Energy", "Valence","Less Than =0, Greater Than =1"]
WEATHER_CONDITIONS = {
    'Clear': [0.6, 0, 6, 1],
    'Rain': [0.3, 0.4, 0],
    'Clouds': [0.5, 0.5, 0],
    'Drizzle': [0.5, 0.6, 0],
    'Atmosphere': [0.5, 0.6, 0],
    'Thunderstorm': [0.5, 0.5, 0]
}

In [3]:
def connect_spotify(config):
    """ connect to spotify
    prerequisites: client_id and client_secret"""

    with open(config) as f:
        c = json.load(f)
        client_id = c["client_id"]
        client_secret = c["client_secret"]
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return spotipy.Spotify(auth_manager=client_credentials_manager)


def connect_database(config):
    """connecting to database
    prerequisites: database, host, name, port, user, pw"""
    with open(config) as f:
        c = json.load(f)
        conn = psycopg2.connect(
            host=c["db_host"],
            dbname=c["db_name"],
            port=c["db_port"],
            user=c["db_user"],
            password=c["db_pw"])
    return conn


def get_db_data(config):
    """select all data from spotify database"""
    conn = connect_database(config=config)
    cur = conn.cursor()
    cur.execute('SELECT * FROM spotify;')
    return cur.fetchall()



def db_to_normal_array(db_res):
    """convert databse to numpy arrary withou preprocessing"""
    l = len(db_res)
    train_X = []
    for i in range(l):
        song_matrix = np.array(list(db_res[i][1:9]))
        train_X.append(song_matrix)
    return train_X


def get_std_playlist(features_arr, uri_arr, db_cont):
    """get standardscaler processed user playlist"""
    db_tmp = db_to_normal_array(db_cont)
    index = []
    flag = 0
    for i in range(len(uri_arr)):
        for item in db_cont:

            if uri_arr[i] == item[0]:
                flag = 1
                index.append(db_cont.index(item))
                break
        if flag == 0:  # not found in db
            db_tmp.append(features_arr[i])
            index.append(len(db_tmp) - 1)
        if flag == 1:
            flag = 0

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(db_tmp)
    test_X = []
    for item in index:
        test_X.append(scaled_features[item])
    test_X = np.array(test_X)

    return test_X


def predict_moods_db(model, db_data):
    db_tmp = db_to_normal_array(db_data)
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(db_tmp)
    test_y = []
    # Feature selection : select feature from 3 to 5  which are speechiness,acousticness,
    # and instrumentalness
    for i in range(len(scaled_features)):
        test_y.append(scaled_features[i][3:6])

    y_pred = model.predict(test_y)  # mods for every entry in spotify table

    db_moods = []
    for i, data in enumerate(db_data):
        new_d = list(data)
        new_d.append(y_pred[i])
        db_moods.append(new_d)
    return db_moods


def input_a_song(sp):  
  # by manually entering songs
  list_of_uris=[]#store uris for songs
  while True:
      user_input=input("Enter the song or 'q': ")
      if user_input=="quit" or user_input == "q" or user_input == '':
        break
      else: 
        result=sp.search(q=user_input,type='track')
        if len(result['tracks']['items']) == 0:
          print(f'Ops, no songs found with that name: {user_input}. Try again!')
        else:
          list_of_uris.append(result['tracks']['items'][0]['uri']) #get the uri of the track(song)
            
  return list_of_uris


# def make_pandas_df(list_of_uris):
#   return pd.DataFrame(list_of_uris, columns=['uri'])
 
##################### KNN #############################
def preprocess_playlist(db_data, playlist, sp_conn):
    """
    :param sp_conn: spotify connection
    :param user_file: file path to csv file with uris :param sp_conn: authorized spotify connection
    :return:
    - list_of_songs: dictionaries of songs
    - song_matrix: np array with songs data ['danceability', 'energy', 'loudness',
                                            'speechiness', 'acousticness','instrumentalness',
                                            'liveness', 'valence']
    - uri_of_playlist: list of uris
    """

    pre_proc_songs = []

    # convert to 2d list and keep only id
    playlist = playlist.to_numpy().tolist()
    playlist_uris = [x[0].split('spotify:track:')[1] for x in playlist
                     if 'spotify:track:' in x[0]] # todo THIS NEEDS SLICE for testing

    uris_found_in_db = []
    # if song is in database add it
    for uri in playlist_uris:
        for song in db_data:
            if uri in song:
                pre_proc_songs.append(song)
                uris_found_in_db.append(uri)
                break

    # else extract data from spotipy + add to database
    uris_not_in_db = list(set(playlist_uris) - set(uris_found_in_db))

    limit = 100  # maxmum 100 at once
    l = len(uris_not_in_db)  # amount of songs
    limitcount = int(l / limit)  # (#songs) /limit

    for j in range(limitcount + 2):
        songleft = l - j * limit
        if songleft > limit:
            features = sp_conn.audio_features(uris_not_in_db[j * limit:j * limit + limit])
            for feat in features:
                result = sp_conn.track(feat['id'])
                song = [feat['id'], feat['danceability'], feat['energy'],
                        feat['loudness'], feat['speechiness'], feat['acousticness'],
                        feat['instrumentalness'], feat['liveness'], feat['valence'],
                        result['popularity'], 'None']  # no genre :/
                pre_proc_songs.append(song)

        elif (songleft <= limit and songleft > 0):
            features = sp_conn.audio_features(uris_not_in_db[j * limit:j * limit + songleft])
            for feat in features:
                result = sp_conn.track(feat['id'])
                song = [feat['id'], feat['danceability'], feat['energy'],
                        feat['loudness'], feat['speechiness'], feat['acousticness'],
                        feat['instrumentalness'], feat['liveness'], feat['valence'],
                        result['popularity'], 'None']  # no genre :/
                pre_proc_songs.append(song)

        else:
            return pre_proc_songs



def preprocess_data_playlist(raw_data, playlist, spotify_conn, rf_model, kn_model):
    """@raw_data: list of songs with their features from db
      @user_playlist: csv with uris
      @return: pre-processed set of songs incl user_song"""
    
    # pick random songs if playlist is too big
    l = len(playlist)
    if l == 0: 
      raise RuntimeError("Playlist is empty")

    if l > 20:
        playlist = playlist.sample(20)
    pre_playlist = preprocess_playlist(raw_data, playlist, spotify_conn) 


    # remove genre for KNN in playlist and db since some genre is None
    playlist = [song[:-1] for song in pre_playlist]
    raw_data_no_genre = [song[:-1] for song in raw_data]
    # predict mood for database and users_playlist
    # rf_model = joblib.load(saved_modelRF)
    playlist_mood = predict_moods_db(rf_model, playlist)
    db_mood = predict_moods_db(kn_model, raw_data_no_genre)
    db_FULL = predict_moods_db(kn_model, raw_data)
    LEN_PLAYLIST = len(playlist)

    # append user's song to the end for later indexing
    for user_song in playlist_mood:
        if user_song in db_mood:
            db_mood.remove(user_song)
        db_mood.append(user_song)

    # get rid of id (since it is string)
    indices = list(range(1, len(db_mood[0])))
    data_no_uri = [itemgetter(*indices)(entry) for entry in db_mood]
    # normalize everything except mood
    #norm_data = normalize(data_no_uri).tolist()
    scaler = StandardScaler()
    norm_data = scaler.fit_transform(data_no_uri).tolist()
  
    pre_data = []
    for n_song, c_song in zip(norm_data, data_no_uri):
        temp_data = copy.deepcopy(n_song[:-1])
        temp_data.append(c_song[-1])
        pre_data.append(temp_data)

    # apply weights (make sure the last value is 1! Because otherwise it will change mood)
    # danceability | energy | loudness | speechiness | acousticness | instrumentalness
    # | liveness | valence | popularity | genre
    weights = [1, 1, 1, 1, 1, 1, 1, 1, 0.3, 1]
    weighted_norm_data = []
    for song_entry in pre_data:
        new_song = []
        for i, value in enumerate(song_entry):
            value = value * weights[i]
            new_song.append(value)
        weighted_norm_data.append(new_song)

    weighted_norm_data = [tuple(weighted_entry) for weighted_entry in weighted_norm_data]
    return weighted_norm_data, len(playlist_mood), db_FULL


def get_recomendation_KNN(cooked_data, num_of_user_songs):
    """recommend top 5 nearest songs to every users song in playlist (last n songs in cooked_data)
        prerequisite: data is normalized and weighted (cooked)"""

    nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(cooked_data)
    distances, indices = nbrs.kneighbors(cooked_data)
    distances, indices = distances.tolist(), indices.tolist()
    # find n-cluster with the user song
    # user songs were appended last, so its indices are:
    user_songs_i = list(range(len(cooked_data) - num_of_user_songs, len(cooked_data)))

    user_songs_neigbors = []  # first element of list = index of user_song (id of song itself), value = list of distances and list of neigbors for that song

    for user_song_i in user_songs_i:
        single_song_neighbors = []
        for ind_list in zip(distances, indices):
            if user_song_i in ind_list[1]:
                single_song_neighbors.append(list(ind_list))
        full = [user_song_i]
        full.append(single_song_neighbors)
        user_songs_neigbors.append(full)

    # select the best cluster -> smallest sum of distances
    # all_songs_best_5 = {} # index of song, best 5 songs indexes as list, # here is sort them by their original song from that the recommendations come
    all_best_songs = []  # here is just a bunch of recommendation independent from original song/ removed users song
    user_songs = []
    for neighbors in user_songs_neigbors:
        song_index = neighbors[0]
        sums = [sum(dist[0]) for dist in neighbors[1]]
        best_5 = neighbors[1][sums.index(min(sums))]
        best_5_i = [ind for ind in best_5[1] if ind != song_index]
        user_songs.append(song_index)
        all_best_songs.append(best_5_i)

    return user_songs, all_best_songs


def search_db(list_idnx, db_data):
    songs_out = []
    for recommendations in list_idnx:
        for indx in recommendations:
            songs_out.append(db_data[indx])
    return songs_out


def filter_by_weather(config, data):
    with open(config) as f:
        config = json.load(f)

    ip_key = config['ip_key']
    weather_key = config['weather_key']

    # get current location
    url = f'http://api.ipstack.com/check?access_key={ip_key}'
    geo_req = json.loads(requests.get(url).text)
    lat = geo_req['latitude']
    lon = geo_req['longitude']

    # get weather of location
    url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={weather_key}'
    # url = f'https://api.openweathermap.org/data/2.5/weather?lat={36.3504}&lon={127.3845}&appid={weather_key}'

    weather_condition = json.loads(requests.get(url).text)['weather'][0]['main']
    filter_cond = WEATHER_CONDITIONS[weather_condition]
    energy, valence, m_l = filter_cond[0], filter_cond[1], filter_cond[2]

    fltrd = []
    for entry in data:
        if m_l == 0:
            if (entry[2] < energy) and (entry[8] < valence):
                fltrd.append(entry)
        if m_l == 1:
            if (entry[2] > energy) and (entry[8] > valence):
                fltrd.append(entry)
   # if all songs were filtered out, give all unfiltered
    if len(fltrd) == 0:
      fltrd = sorted(data, key=lambda x: x[-3], reverse=True)
      fif_perc = len(fltrd) // 2 # output 50% of unfiltered data with highest popularity
      fltrd = fltrd[:fif_perc]
    return weather_condition, fltrd


def visualize(data):
    recommendations_result = []
    for recomendated_data in data:
        res = sp.track(recomendated_data[0])
        artist = res['artists'][0]['name']
        song = res['name']
        recommendations_result.append([recomendated_data[0], artist, song,
                                       recomendated_data[-3], recomendated_data[-2], MOOD[recomendated_data[-1]]])

    df = pd.DataFrame(recommendations_result, columns=['URI', 'Artist', 'Song', 'Popularity', 'Genre', 'Mood'])
    df.set_index('URI', inplace=True)
    df.sort_values(by=['Popularity'], inplace=True, ascending=False)
    return df

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
# Prerequisite: config files

root = '/content/gdrive/My Drive/code/'
config = root + 'config.json'
sp = connect_spotify(config)
db_data = cur = get_db_data(config)
saved_modelRF = root + 'RFmodel.pkl'
km_model_path = root + 'Kstd6.pkl'
# load models
km_model = joblib.load(km_model_path)
rf_model = joblib.load(saved_modelRF)

#### MAIN 

while True:
    print("""Choose input:\n
                    a - csv with spotify uris
                    b - write song names by yourself
                    q - Quit
                    """)
    answer = input("Enter your choice: ").strip()
    if answer not in ['a', 'b', 'q']:
        print("Please choose between a, b or q")
    else:
        break

user_playlist = None
if answer == "q":
  quit() #for python
  #!kill -9 -1 # for colab
elif answer == 'a':
  filename = input("Provide the csv file name: ").strip()
  user_playlist = pd.read_csv(root + filename) 
else:
  uris = input_a_song(sp)
  user_playlist = pd.DataFrame(uris, columns=['uri'])

# preprocess data format and get the audio features, normalization and weights.
data_playlist_pre_processed, num_user_songs, db_full = preprocess_data_playlist(db_data, user_playlist, sp, rf_model, km_model)
# recommend with the preprocessed playlist data
given_songs, recommendations = get_recomendation_KNN(data_playlist_pre_processed, num_user_songs)

# find recommended songs in database by their index
result_data = search_db(recommendations, db_full)
# filter recommendations by weather conditions
# print(f'Before filtering: {len(result_data)}')
weather, result_fltrd = filter_by_weather(config, result_data)
# print(f'After filtering: {len(result_fltrd)}')

# visualize results
print(f'These are the songs recommendations for you. BTW, it is {weather} outside!') 
table = visualize(result_fltrd)
table


Choose input:

                    a - csv with spotify uris
                    b - write song names by yourself
                    q - Quit
                    
Enter your choice: STAY
Please choose between a, b or q
Choose input:

                    a - csv with spotify uris
                    b - write song names by yourself
                    q - Quit
                    
Enter your choice: b
Enter the song or 'q': STAY
Enter the song or 'q': I like you
Enter the song or 'q': First Class
Enter the song or 'q': Woman
Enter the song or 'q': q
These are the songs recommendations for you. BTW, it is Clear outside!


,Artist,Song,Popularity,Genre,Mood
URI,,,,,
6MWtB6iiXyIwun0YzU6DFP,Post Malone,Wow.,99,Rap,Happy
6JQvHMSaR8flFsN53ypOfP,Lil Mosey,Noticed,87,Rap,Sad
4hQ6UGyWQIGJmHSo0J88JW,Selena Gomez,Back To You - From 13 Reasons Why – Season 2 S...,84,Dance,Energetic
1XRgIKC5TPwo7nWGyKqgG0,Gucci Mane,I Get the Bag (feat. Migos),79,Hip-Hop,Happy
2lYTJK94hb0fd1LQtb6Dhk,David Guetta,Don't Leave Me Alone (feat. Anne-Marie),79,Dance,Energetic
11IzgLRXV7Cgek3tEgGgjw,Queen,Under Pressure - Remastered 2011,77,Rock,Sad
1RagpNpkD3KcjXMLGMe99V,Wiz Khalifa,Fr Fr (feat. Lil Skies),76,Hip-Hop,Happy
514rhnksEwHUh6LxXsQ4Y9,Selena Gomez,Back To You,76,Dance,Energetic
